In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import copy
np.set_printoptions(threshold = np.inf)

In [ ]:
"""
@params
    array = state grid we constructed
    row_number = #rows of state grid
    column_number = #columns of state grid
    episodes = #episodes
    steps = #steps within episodes
    alpha = learning rate
    gama = discount factor

"""

def applyQlearning(array,row_number,column_number,episodes,steps,alpha,gama):
    epsilon_q = 0.1
    state_number = row_number * column_number
    last_row_index = row_number - 1
    last_column_index = column_number - 1
    action_number = 4
    q_table = np.random.rand(state_number, action_number)
    # terminal state initialization
    q_table[state_number - 1][0] = 0
    q_table[state_number - 1][1] = 0
    q_table[state_number - 1][2] = 0
    q_table[state_number - 1][3] = 0
    rewardArray = []
    episodeArray = []
    #   averageRewardArray = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    windowing_average_samples = 40
    averageRewardArray = np.zeros(windowing_average_samples)
    for i in range(episodes):  # loop for each episode
        step = 0
        #     if (i % 50 == 0 and epsilonNew > 0.01):
         #   epsilonNew = epsilonNew - 0.01
        current_state = array[3][0]  # initialize S
        count = 0
        totalReward = 0
        episodeArray.append(i)
        while (count in range(steps) and current_state.isTerminal(last_row_index) == 0):
            step = step + 1
            count = count + 1
            # choose action A from S using policy derived from Q
            action = current_state.eGreedy(q_table,epsilon_q, last_row_index,last_column_index)
            # take action A, observe R,S'
            next_state = current_state.takeAction(array, action, q_table,last_row_index,last_column_index,epsilon_q)
            reward_got = next_state.getReward()
            totalReward = totalReward + reward_got
            actionMax = next_state.ePureGreedy(q_table,last_row_index,last_column_index)
            q_table_max = q_table[(next_state.row * column_number) + next_state.column][actionMax]
            q_table_current = q_table[(current_state.row * column_number) + current_state.column][action]
            q_table[(current_state.row * column_number) + current_state.column][action] = q_table_current +\
                                            alpha * (reward_got + (gama * q_table_max) - q_table_current)  # update
            current_state = next_state
        averageRewardArray = shiftAndAddArray(averageRewardArray, totalReward)
        valueToDraw = np.sum(averageRewardArray) / np.count_nonzero(averageRewardArray)
        rewardArray.append(valueToDraw)
    print("Q TRAINING IS OVER")
    episodeArray = episodeArray[10:]
    rewardArray = rewardArray[10:]
    drawPath(array, q_table,last_row_index,last_column_index,epsilon_q)
    plt.plot(episodeArray, rewardArray)
    plt.legend(['SARSA','Q learning'])
    print(averageRewardArray)
    plt.title(f'Smoothed by averaging {windowing_average_samples} successive episodes')
    # limitting the overall boundary to have a nice vision
    plt.ylim((-120, 0))
    plt.show()

In [ ]:
#  this is for the test after the training
def drawPath(array,Qvalue,last_row_index, last_column_index,epsilon):
    current_state = array[last_row_index][0]  # initial state
    greedyAction = current_state.ePureGreedy(Qvalue,last_row_index,last_column_index)  # action is determined
    while(current_state.isTerminal(last_row_index) == 0):
        next_state = current_state.takeAction(array,greedyAction,Qvalue,last_row_index, last_column_index,epsilon)
        next_state.changeSign("X")
        followingAction = next_state.ePureGreedy(Qvalue,last_row_index, last_column_index)
        current_state = next_state
        greedyAction = followingAction
    display2D(array)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import random

def display2D(array):
    for row in array:
        for column in row:
            print(column.getSign(), end=" ")
        print(end="\n")

def displayRewards(array):
    for row in array:
        for column in row:
            print(column.getReward(), end=" ")
        print(end='\n')

def handleRewards(array, last_row, column):
    for i in range(column - 2):
        array[last_row][i + 1].setReward(-100)

def shiftAndAddArray(array_to_shift, new_element):
    for i in range(len(array_to_shift) - 1):
        array_to_shift[i] = array_to_shift[i + 1]
    array_to_shift[len(array_to_shift) - 1] = new_element
    return array_to_shift

class States:
    def __init__(self, sign, reward, row, column):
        self.sign = sign
        self.reward = reward
        self.row = row
        self.column = column

    def getReward(self):
        return self.reward

    def getSign(self):
        return self.sign

    def changeSign(self, new_sign):
        self.sign = new_sign

    def setReward(self, new_reward):
        self.reward = new_reward

    def eGreedy(self, q_value, epsilon, last_row_index, last_column_index):
        new_action_list = [0, 1, 2, 3]  # 0 for right, 1 for down, 2 for up, 3 for left

        if self.row == 0 and self.column == 0:
            new_action_list.remove(3)
            new_action_list.remove(2)
        elif self.row == last_row_index and self.column == 0:
            new_action_list.remove(3)
            new_action_list.remove(1)
        elif self.row == 0 and self.column == last_column_index:
            new_action_list.remove(0)
            new_action_list.remove(2)
        elif self.row == last_row_index and self.column == last_column_index:
            new_action_list.remove(0)
            new_action_list.remove(1)
        elif self.row == 0:
            new_action_list.remove(2)
        elif self.row == last_row_index:
            new_action_list.remove(1)
        elif self.column == 0:
            new_action_list.remove(3)
        elif self.column == last_column_index:
            new_action_list.remove(0)

        probability = random.uniform(0, 1)

        if probability < 1 - epsilon:
            return self.findMaxIndex(q_value, new_action_list, last_row_index, last_column_index)
        else:
            return random.choice(new_action_list)

    def ePureGreedy(self, q_value, last_row_index, last_column_index):
        new_action_list = [0, 1, 2, 3]

        if self.row == 0 and self.column == 0:
            new_action_list.remove(3)
            new_action_list.remove(2)
        elif self.row == last_row_index and self.column == 0:
            new_action_list.remove(3)
            new_action_list.remove(1)
        elif self.row == 0 and self.column == last_column_index:
            new_action_list.remove(0)
            new_action_list.remove(2)
        elif self.row == last_row_index and self.column == last_column_index:
            new_action_list.remove(0)
            new_action_list.remove(1)
        elif self.row == 0:
            new_action_list.remove(2)
        elif self.row == last_row_index:
            new_action_list.remove(1)
        elif self.column == 0:
            new_action_list.remove(3)
        elif self.column == last_column_index:
            new_action_list.remove(0)

        return self.findMaxIndex(q_value, new_action_list, last_row_index, last_column_index)

    def takeAction(self, array, action, q_value, last_row_index, last_column_index, epsilon):
        if self.row == 0 and action == 2:
            new_action = self.eGreedy(q_value, epsilon, last_row_index, last_column_index)
            return self.takeAction(array, new_action, q_value, last_row_index, last_column_index, epsilon)
        elif self.row == last_row_index and action == 1:
            new_action = self.eGreedy(q_value, epsilon, last_row_index, last_column_index)
            return self.takeAction(array, new_action, q_value, last_row_index, last_column_index, epsilon)
        elif self.column == last_column_index and action == 0:
            new_action = self.eGreedy(q_value, epsilon, last_row_index, last_column_index)
            return self.takeAction(array, new_action, q_value, last_row_index, last_column_index, epsilon)
        elif self.column == 0 and action == 3:
            new_action = self.eGreedy(q_value, epsilon, last_row_index, last_column_index)
            return self.takeAction(array, new_action, q_value, last_row_index, last_column_index, epsilon)
        else:
            if action == 0:
                next_state = array[self.row][self.column + 1]
            elif action == 1:
                next_state = array[self.row + 1][self.column]
            elif action == 2:
                next_state = array[self.row - 1][self.column]
            elif action == 3:
                next_state = array[self.row][self.column - 1]
        return next_state

    def findMaxIndex(self, q_value, action_list, last_row_index, last_column_index):
        max_index = np.argmax(q_value[self.row * 12 + self.column])
        return max_index

    def isTerminal(self, last_row_index):
        if self.row == last_row_index and self.column > 0:
            return 1
        else:
            return 0

def applyQlearning(array, row_number, column_number, episodes, steps, alpha, gamma, render=False):
    epsilon_q = 0.1
    state_number = row_number * column_number
    last_row_index = row_number - 1
    last_column_index = column_number - 1
    action_number = 4
    q_table = np.zeros((state_number, action_number))

    reward_array = []
    episode_array = []
    windowing_average_samples = 40
    average_reward_array = np.zeros(windowing_average_samples)

    env = gym.make('CliffWalking-v0')  # Create the CliffWalking environment

    for i in range(episodes):
        step = 0
        current_state = States('S', 0, 3, 0)  # Assuming starting state, adjust accordingly
        count = 0
        total_reward = 0
        episode_array.append(i)

        while count in range(steps) and not current_state.isTerminal(last_row_index):
            step += 1
            count += 1

            action = current_state.eGreedy(q_table, epsilon_q, last_row_index, last_column_index)
            next_state_info = current_state.takeAction(array, action, q_table, last_row_index, last_column_index, epsilon_q)
            next_state = States(*next_state_info)

            reward_got = next_state.getReward()
            total_reward += reward_got
            action_max = next_state.ePureGreedy(q_table, last_row_index, last_column_index)
            q_table_max = np.max(q_table[(next_state.row * column_number) + next_state.column, :])
            q_table_current = q_table[(current_state.row * column_number) + current_state.column, action]

            q_table[(current_state.row * column_number) + current_state.column, action] = (
                q_table_current + alpha * (reward_got + gamma * q_table_max - q_table_current)
            )

            current_state = next_state

            if render:
                env.render()

        average_reward_array = shiftAndAddArray(average_reward_array, total_reward)
        value_to_draw = np.sum(average_reward_array) / np.count_nonzero(average_reward_array)
        reward_array.append(value_to_draw)

        # Display and handle rewards
        displayRewards(array)
        handleRewards(array, last_row_index, column_number)

    print("Q TRAINING IS OVER")
    episode_array = episode_array[10:]
    reward_array = reward_array[10:]

    plt.plot(episode_array, reward_array)
    plt.legend(['SARSA', 'Q learning'])
    plt.title(f'Smoothed by averaging {windowing_average_samples} successive episodes')
    plt.ylim((-120, 0))
    plt.show()

    env.close()

# Example usage with your state grid
# Replace the following lines with your specific state grid, row_number, column_number, etc.
array = [[States('O', -1, i, j) for j in range(4)] for i in range(4)]
row_number = 4
column_number = 4
episodes = 500
steps = 100
alpha = 0.1
gamma = 0.9

applyQlearning(array, row_number, column_number, episodes, steps, alpha, gamma, render=True)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import random
import copy

# Your States class implementation here

def display2D(array):
    for row in array:
        for column in row:
            print(column.getSign(), end=" ")
        print(end="\n")

def displayRewards(array):
    for row in array:
        for column in row:
            print(column.getReward(), end=" ")
        print(end='\n')

def handleRewards(array, last_row, column):
    for i in range(column - 2):
        array[last_row][i + 1].setReward(-100)

def shiftAndAddArray(array_to_shift, new_element):
    for i in range(len(array_to_shift) - 1):
        array_to_shift[i] = array_to_shift[i + 1]
    array_to_shift[len(array_to_shift) - 1] = new_element
    return array_to_shift

def applyQlearning(array, row_number, column_number, episodes, steps, alpha, gamma, render=False):
    epsilon_q = 0.1
    state_number = row_number * column_number
    last_row_index = row_number - 1
    last_column_index = column_number - 1
    action_number = 4
    q_table = np.zeros((state_number, action_number))

    reward_array = []
    episode_array = []
    windowing_average_samples = 40
    average_reward_array = np.zeros(windowing_average_samples)

    env = gym.make('CliffWalking-v0')

    for i in range(episodes):
        step = 0
        current_state = States('S', 0, 3, 0)
        count = 0
        total_reward = 0
        episode_array.append(i)

        while count in range(steps) and not current_state.isTerminal(last_row_index):
            step += 1
            count += 1

            action = current_state.eGreedy(q_table, epsilon_q, last_row_index, last_column_index)
            next_state_info = current_state.takeAction(array, action, q_table, last_row_index, last_column_index, epsilon_q)
            next_state = States(*next_state_info)

            reward_got = next_state.getReward()
            total_reward += reward_got
            action_max = next_state.ePureGreedy(q_table, last_row_index, last_column_index)
            q_table_max = np.max(q_table[(next_state.row * column_number) + next_state.column, :])
            q_table_current = q_table[(current_state.row * column_number) + current_state.column, action]

            q_table[(current_state.row * column_number) + current_state.column, action] = (
                q_table_current + alpha * (reward_got + gamma * q_table_max - q_table_current)
            )

            current_state = next_state

            if render:
                env.render()

        average_reward_array = shiftAndAddArray(average_reward_array, total_reward)
        value_to_draw = np.sum(average_reward_array) / np.count_nonzero(average_reward_array)
        reward_array.append(value_to_draw)

        # Display and handle rewards
        displayRewards(array)
        handleRewards(array, last_row_index, column_number)

    print("Q TRAINING IS OVER")
    episode_array = episode_array[10:]
    reward_array = reward_array[10:]

    plt.plot(episode_array, reward_array)
    plt.legend(['SARSA', 'Q learning'])
    plt.title(f'Smoothed by averaging {windowing_average_samples} successive episodes')
    plt.ylim((-120, 0))
    plt.show()

    env.close()

def applySarsa(array, row_number, column_number, episodes, steps, alpha, gamma, render=False):
    epsilon_q = 0.1
    state_number = row_number * column_number
    last_row_index = row_number - 1
    last_column_index = column_number - 1
    action_number = 4
    q_table = np.zeros((state_number, action_number))

    reward_array = []
    episode_array = []
    windowing_average_samples = 40
    average_reward_array = np.zeros(windowing_average_samples)

    env = gym.make('CliffWalking-v0')

    for i in range(episodes):
        step = 0
        current_state = States('S', 0, 3, 0)
        count = 0
        total_reward = 0
        episode_array.append(i)

        while count in range(steps) and not current_state.isTerminal(last_row_index):
            step += 1
            count += 1

            action = current_state.eGreedy(q_table, epsilon_q, last_row_index, last_column_index)
            next_state_info = current_state.takeAction(array, action, q_table, last_row_index, last_column_index, epsilon_q)
            next_state = States(*next_state_info)

            reward_got = next_state.getReward()
            total_reward += reward_got
            next_action = next_state.eGreedy(q_table, epsilon_q, last_row_index, last_column_index)
            q_table_max = np.max(q_table[(next_state.row * column_number) + next_state.column, :])
            q_table_current = q_table[(current_state.row * column_number) + current_state.column, action]

            q_table[(current_state.row * column_number) + current_state.column, action] = (
                q_table_current + alpha * (reward_got + gamma * q_table_max - q_table_current)
            )

            current_state = next_state

            if render:
                env.render()

        average_reward_array = shiftAndAddArray(average_reward_array, total_reward)
        value_to_draw = np.sum(average_reward_array) / np.count_nonzero(average_reward_array)
        reward_array.append(value_to_draw)

        # Display and handle rewards
        displayRewards(array)
        handleRewards(array, last_row_index, column_number)

    print("SARSA TRAINING IS OVER")
    episode_array = episode_array[10:]
    reward_array = reward_array[10:]

    plt.plot(episode_array, reward_array)
    plt.legend(['SARSA', 'Q learning'])
    plt.title(f'Smoothed by averaging {windowing_average_samples} successive episodes')
    plt.ylim((-120, 0))
    plt.show()

    env.close()

def main():
    episode_number = 600
    steps = 500
    gamma = 0.9
    alpha = 0.25
    row_number = 4
    column_number = 12

    stateGrid = [[States('O',-1,y,x) for x in range (column_number)] for y in range(row_number)]

    display2D(stateGrid)
    print("OK HERE WE STOP")
    stateGrid[row_number - 1][0].changeSign('S')
    stateGrid[row_number - 1][column_number - 1].changeSign('F')
    for i in range(10):
        stateGrid[row_number - 1][i+1].changeSign('C')

    display2D(stateGrid)
    handleRewards(stateGrid, row_number - 1, column_number)
    displayRewards(stateGrid)
    print("CONSTRUCTED")

    stateGrid2 = copy.deepcopy(stateGrid)
    applySarsa(stateGrid, row_number, column_number, episode_number, steps, alpha, gamma)
    applyQlearning(stateGrid2, row_number, column_number, episode_number, steps, alpha, gamma)

if __name__ == "__main__":
    main()
